# Generative AI and Multi-Modal Agents in AWS: The Key to Unlocking New Value in Financial Markets

# Preparation

### Install the necessary libraries.

In [ ]:
%%writefile demo-requirements.txt
anthropic==0.2.10
boto3==1.26.163
langchain==0.0.213
PyAthena[SQLAlchemy]==2.25.2
sqlalchemy==1.4.47
PyPortfolioOpt

In [ ]:
!pip install -r demo-requirements.txt --quiet

please ignore the error message when installing packages

In [ ]:
# modify the region
# Make sure it's a region that supports Kendra. https://aws.amazon.com/about-aws/whats-new/2021/06/amazon-kendra-adds-support-for-new-aws-regions/

region = 'us-east-2' 

In [ ]:
# This snippet get the key from secret manager
import boto3
from botocore.exceptions import ClientError
import json


def get_secret():

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e

    # Decrypts secret using the associated KMS key.
    secret = get_secret_value_response['SecretString']

    return json.loads(secret)



# Use one of the following methods to get Anthropic API Key.

Method 1: Retrieve the key from AWS Secrets Manager. Uncomment the code if you use this method.

In [ ]:
# secret_name = "anthropic_key" #Modify the secret name
# ANTHROPIC_API_KEY = get_secret()['anthropic_key']

Method 2: Directly enter the Anthropic API Key. Uncomment the code if you use this method.

In [ ]:
# #ANTHROPIC_API_KEY = ''

### Get resource names

The CloudFormation stack created the infrastructure for this application, such as S3 buckets and Lambda functions. We will get the names/ids of these resources. 

Modify the CFN_STACK_NAME based on the name you used when creating the CloudFormation stack.

In [ ]:
# modify the stack name to match the name of your CloudFormation stack

CFN_STACK_NAME = "multimodal-cf"

In [ ]:
import boto3
from typing import List


stacks = boto3.client('cloudformation').list_stacks()
stack_found = CFN_STACK_NAME in [stack['StackName'] for stack in stacks['StackSummaries']]


def get_cfn_outputs(stackname: str) -> List:
    cfn = boto3.client('cloudformation')
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs



# this function extracts the bucket name from S3 uri.
# For example, the bucket name is 'my_bucket' based on the URI "s3://my_bucket/"
def get_bucket_name(s3_uri):
    bucket_name = s3_uri.split("/")[2]
    return bucket_name


if stack_found is True:
    outputs = get_cfn_outputs(CFN_STACK_NAME)
    glue_db_name = outputs['stockpricesdb']
    kendra_index_id = outputs['KendraIndexId']
    audio_transcripts_source_bucket = get_bucket_name(outputs['AudioSourceBucket'])
    textract_source_bucket = get_bucket_name(outputs['PDFSourceBucket'])
    query_staging_bucket = get_bucket_name(outputs['QueryStagingBucket'])
    stock_data_source_bucket = get_bucket_name(outputs['StockDataSourceBucket'])
    multimodal_output_bucket = get_bucket_name(outputs['MultimodalOutputBucket'])
    print (f"glue_db_name is {glue_db_name}.")
    print (f"kendra_index_id is {kendra_index_id}.")
    print (f"audio_transcripts_source_bucket is {audio_transcripts_source_bucket}.")
    print (f"textract_source_bucket is {textract_source_bucket}.")
    print (f"query_staging_bucket is {query_staging_bucket}.")
    print (f"stock_data_source_bucket is {stock_data_source_bucket}.")
    print (f"multimodal_output_bucket is {multimodal_output_bucket}.")
else:
    print("Recheck our cloudformation stack name")

### Upload the files to the S3 buckets

Next, we will upload the documents to the S3 buckets created by CloudFormation.

In [ ]:
import logging
import boto3
from botocore.exceptions import ClientError
import os


def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
        print (f'Uploaded {file_name} to S3 bucket {bucket}.')
    except ClientError as e:
        logging.error(e)
        return False
    return True



upload_file('files/Amazon-10K-2022-EarningsReport.pdf', textract_source_bucket)
upload_file('files/Amazon-10Q-Q1-2023-QuaterlyEarningsReport.pdf', textract_source_bucket)
upload_file('files/Amazon-Quarterly-Earnings-Report-Q1-2023-Full-Call-v1.mp3', audio_transcripts_source_bucket)
upload_file('./files/stock_prices.csv', stock_data_source_bucket)

### Create a table in Amazon Athena
We will store the stock data in Athena for querying. The stock data has the following format:

date|AAAA|FF|BBB|ZZZZ|...

AAAA, etc., are fake stock symbols.



First, drop the existing table as we will create a new one. Copy and past the query in the Athena Query Editor.

```
DROP TABLE `stock_prices`;
```

In [ ]:
print (f"stock_data_source_bucket is {stock_data_source_bucket}")

In [ ]:
import boto3
client = boto3.client('athena')

def query_athena(query):
    response = client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': 'blog-stock-prices-db'
        },
        ResultConfiguration={
            'OutputLocation': f's3://{query_staging_bucket}/',
        },
        WorkGroup='primary'
    )
    print(response)

Replace the existing table with a new table that imports the csv file

In [ ]:
drop_table_query='DROP TABLE `stock_prices`;'
query_athena(drop_table_query)

In [ ]:
create_table_query=f"""
CREATE EXTERNAL TABLE IF NOT EXISTS `blog-stock-prices-db`.stock_prices ( 
    date string, 
    AAAA double, 
    FF double, 
    BBBB double, 
    ZZZZ double, 
    GG double, 
    DDD double, 
    WWW double, 
    CCC double, 
    GGMM double, 
    TTT double, 
    UUU double, 
    SSSS double, 
    XXX double, 
    RRR double, 
    YYY double, 
    MM double, 
    PPP double, 
    JJJ double, 
    SSXX double
) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES ('separatorChar' = ',', 'quoteChar' = '\\\"', 'escapeChar' = '\\\\')
LOCATION 's3://{stock_data_source_bucket}/'
TBLPROPERTIES ('skip.header.line.count'='1')
"""
query_athena(create_table_query)

# Create tools

In the following section, we will define various tools for the agent. The tools include:
    
* Stocks Querying Tool to query S&P stocks data using Amazon Athena and SQL Alchemy.
* Portfolio Optimization Tool that builds a portfolio based on the chosen stocks.
* Financial Information Lookup Tool to search for financial earnings information stored in multi-page pdf files using Amazon Kendra.
* Python Calculation Tool that can be used to do mathematical calculations.
* Sentiment Analysis Tool to identify and score sentiments on a topic using Amazon Comprehend.
* Detect Phrases Tool to find key phrases in recent quarterly reports using Amazon Comprehend.
* Text Extraction Tool to convert pdf version of quarterly reports to text files using Amazon Textract.
* Transcribe Audio Tool to convert audio recordings to text files using Amazon Transcribe.



In [ ]:
import json
import boto3

import sqlalchemy
from sqlalchemy import create_engine

from langchain.docstore.document import Document
from langchain import PromptTemplate,SQLDatabase, SQLDatabaseChain, LLMChain
from langchain.prompts.prompt import PromptTemplate

from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chat_models import ChatAnthropic
from langchain.chains.api import open_meteo_docs

import pandas as pd
import datetime
import pandas as pd
from functools import reduce
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from langchain.tools import tool
from langchain.tools.base import StructuredTool
from typing import Optional
from langchain.tools import BaseTool
from typing import List, Optional

from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.agents.tools import Tool
from langchain import LLMMathChain

from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory

from typing import Dict
import time
import uuid
import boto3


In [ ]:
# Connect to an LLM

llm = ChatAnthropic(temperature=0, anthropic_api_key=ANTHROPIC_API_KEY, max_tokens_to_sample = 512)

## Create Stocks Querying Tool

In [ ]:
# Modify the following parameters as needed
table = 'stock_prices'

connathena=f"athena.{region}.amazonaws.com" 
portathena='443' #Update, if port is different
schemaathena=glue_db_name #from user defined params
s3stagingathena=f's3://{query_staging_bucket}/athenaresults/'#from cfn params
wkgrpathena='primary'#Update, if workgroup is different

##  Create the athena connection string
connection_string = f"awsathena+rest://@{connathena}:{portathena}/{schemaathena}?s3_staging_dir={s3stagingathena}&work_group={wkgrpathena}"

##  Create the athena  SQLAlchemy engine
engine_athena = create_engine(connection_string, echo=False)
dbathena = SQLDatabase(engine_athena)

<!-- Define the SQL query function. The input of this function will be a prompt in plain English, such as "What is the size of this table?" The function will translate the prompt into a SQL query and run the query using the Athena database.
 -->

In [ ]:
# define the prompt template for the Stock Querying Tool

_DEFAULT_TEMPLATE = """
    Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
    
    Do not append 'Query:' to SQLQuery.
    
    For example, if I want to get stock price information for aaaa, gg and ddd, the query should be :
    
    SELECT date, aaaa, gg, ddd FROM "blog-stock-prices-db"."stock_prices" order by date asc
    
    Display SQLResult after the query is run in plain english that users can understand. 
    

    Provide answer in simple english statement.
 
    Only use the following tables:

    {table_info}
    If someone asks about closing price of a stock, it should be the last price at which a stock trades during a regular trading session.
    
    Question: {input}
    
    Provide answer to the input question based on the query results.  
    """
    

In [ ]:
def run_query(query):

    PROMPT_sql = PromptTemplate(
        input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
    )
    
    db_chain = SQLDatabaseChain.from_llm(llm, dbathena, prompt=PROMPT_sql, verbose=True, return_intermediate_steps=False)
    response=db_chain.run(query)
    
    return response

Test the run_query tool with a simple question.

In [ ]:
# test the run_query tool
run_query('What is the size of the table?')

## Create Portfolio Optimization Tool 

In [ ]:
import pandas as pd
import datetime
import pandas as pd
from functools import reduce
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from langchain.tools import tool
from langchain.tools.base import StructuredTool
from typing import Optional
from langchain.tools import BaseTool
from typing import List, Optional


class OptimizePortfolio(BaseTool):
    
    import pandas as pd
    
    name = "Portfolio Optimization Tool"
    description = """
        use this tool when you need to build optimal portfolio. 
        The output results tell you the allocation of your money on each stock.
        The stock_ls should be a list of stock symbols, such as ['WWW', 'DDD', 'AAAA'].
        """

    
    def _run(self, stock_ls: List):
        
        import boto3
        import pandas as pd
        from pyathena import connect
        
        # Establish connection to Athena
        session = boto3.Session(region_name=region)
        athena_client = session.client('athena')

        # Execute query

        stock_seq = ', '.join(stock_ls)
        query = f'SELECT date, {stock_seq} from "{glue_db_name}"."{table}"'
        print (f'query:{query}')
        cursor = connect(s3_staging_dir=f's3://{query_staging_bucket}/athenaresults/', region_name=region).cursor()
        cursor.execute(query)

        # Fetch results
        rows = cursor.fetchall()

        # Convert to Pandas DataFrame
        df = pd.DataFrame(rows, columns=[column[0] for column in cursor.description])

        # Set "Date" as the index and parse it as a datetime object
        df.set_index("date", inplace=True)
        df.index = pd.to_datetime(df.index, format = '%Y-%m-%d')
        
        mu = expected_returns.mean_historical_return(df)
        S = risk_models.sample_cov(df)

        # Optimize for maximal Sharpe ratio
        ef = EfficientFrontier(mu, S)
        weights = ef.max_sharpe()
        ef.portfolio_performance(verbose=True)

        cleaned_weights = ef.clean_weights()
        print (f'cleaned_weights are {dict(cleaned_weights)}')

        ef.portfolio_performance(verbose=True)

        #Finally, let’s convert the weights into actual allocations values (i.e., how many of each stock to buy). For our allocation, let’s consider an investment amount of $100,000:

        from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices


        latest_prices = get_latest_prices(df)

        da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=10000)
        allocation, leftover = da.greedy_portfolio()
        print("Discrete allocation:", allocation)
        print("Funds remaining: ${:.2f}".format(leftover))
        print (allocation)
        return cleaned_weights

    def _arun(self, stock_ls: int):
        raise NotImplementedError("This tool does not support async")

# Define tools that use Amazon Comprehend

In [ ]:
def SentimentAnalysis(inputString):
    print(inputString)
    lambda_client = boto3.client('lambda')
    lambda_payload = {"inputString:"+inputString}
    response=lambda_client.invoke(FunctionName='FSI-SentimentDetecttion',
                        InvocationType='RequestResponse',
                     Payload=json.dumps(inputString))
    output=json.loads(response['Payload'],read().decode())
    return output['body']

def DetectKeyPhrases(inputString):
    print(inputString)
    lambda_client = boto3.client('lambda')
    lambda_payload = {"inputString:"+inputString}
    response=lambda_client.invoke(FunctionName='FSI-KeyPhrasesDetection',
                        InvocationType='RequestResponse',
                     Payload=json.dumps(inputString))
    output=json.loads(response['Payload'],read().decode())
    return output['body']


# Define tool that uses AWS Textract

In [ ]:
def IntiateTextExtractProcessing(inputString):
    print(inputString)
    lambda_client = boto3.client('lambda')
    lambda_payload = {"inputString:"+inputString}
    response=lambda_client.invoke(FunctionName='FSI-TextractAsyncInvocationFunction',
                        InvocationType='RequestResponse',
                     Payload=json.dumps(inputString))
    print(response['Payload'].read())
    return response

# Define tool that uses Amazon Transcribe

In [ ]:
def TranscribeAudio(inputString):
    print(inputString)
    lambda_client = boto3.client('lambda')
    lambda_payload = {"inputString:"+inputString}
    response=lambda_client.invoke(FunctionName='FSI-Transcribe',
                        InvocationType='RequestResponse',
                     Payload=json.dumps(inputString))
    print(response['Payload'].read())
    return response

# Create Financial Information Lookup Tool 

Kendra helps you find faster with intelligent enterprise search powered by machine learning. We will use Kendra to find answers in *Amazon-10K-2022-EarningsReport.pdf*, *Amazon-10Q-Q1-2023-QuaterlyEarningsReport.pdf* and trasncriptions of *Amazon-Quarterly-Earnings-Report-Q1-2023-Full-Call-v1.mp3*.

Sample question: “What’s Amazon’s unearned revenue from AWS and Prime memberships as of December 31, 2022? What is the profitability ratio as of December 31, 2022"

First, we to do two pre-processing steps to extract the text of the PDF files. This process takes about 15 minutes to finish.

In [ ]:
IntiateTextExtractProcessing('process')

Next, we need to transcribe the .mpd audio file. This also takes about 15 minutes.

In [ ]:
TranscribeAudio('process')

The above process takes about 15 minutes. The code below will check the S3 bucket every minute. If the process is completed, the code will show "The S3 bucket contains all the necessary output.", and please proceed to the next cell. 

In [ ]:
def list_s3_files_using_client(bucket):
    """
    This functions list all files in s3 bucket.
    :return: None
    """
    s3_client=boto3.client("s3")
    bucket_name=bucket
    response=s3_client.list_objects_v2(Bucket=bucket_name)
    files=response.get("Contents")
    output_list=[]
    for file in files:
        output_list.append(file['Key'])
    return output_list

correct_list = [
    'audiooutputs/Amazon-Quarterly-Earnings-Report-Q1-2023-Full-Call-v1.txt',
    'pdfoutputs/Amazon-10K-2022-EarningsReport.txt',
    'pdfoutputs/Amazon-10Q-Q1-2023-QuaterlyEarningsReport.txt'
]


s3_files_list=list_s3_files_using_client(multimodal_output_bucket)
check =  all(item in s3_files_list for item in correct_list)
 
while not check:
    time.sleep(30)
    s3_files_list=list_s3_files_using_client(multimodal_output_bucket)
    check =  all(item in s3_files_list for item in correct_list)
else:
    print ('The S3 bucket contains all the necessary output. Please proceed.')


Next, we are going to sync Amazon Kendra with the text files. 

In [ ]:
import boto3
from typing import List


stacks = boto3.client('cloudformation').list_stacks()
stack_found = CFN_STACK_NAME in [stack['StackName'] for stack in stacks['StackSummaries']]


def get_cfn_outputs(stackname: str) -> List:
    cfn = boto3.client('cloudformation')
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs



# this function extracts the bucket name from S3 uri.
# For example, the bucket name is 'my_bucket' based on the URI "s3://my_bucket/"
def get_bucket_name(s3_uri):
    bucket_name = s3_uri.split("/")[2]
    return bucket_name


if stack_found is True:
    outputs = get_cfn_outputs(CFN_STACK_NAME)
    glue_db_name = outputs['stockpricesdb']
    kendra_index_id = outputs['KendraIndexId']
    kendra_data_source_id = outputs['KendraDataSourceId']
    audio_transcripts_source_bucket = get_bucket_name(outputs['AudioSourceBucket'])
    textract_source_bucket = get_bucket_name(outputs['PDFSourceBucket'])
    query_staging_bucket = get_bucket_name(outputs['QueryStagingBucket'])
    stock_data_source_bucket = get_bucket_name(outputs['StockDataSourceBucket'])
    multimodal_output_bucket = get_bucket_name(outputs['MultimodalOutputBucket'])
    
    print (f"glue_db_name is {glue_db_name}.")
    print (f"kendra_index_id is {kendra_index_id}.")
    print (f"kendra_data_source_id is {kendra_data_source_id}.")
    print (f"audio_transcripts_source_bucket is {audio_transcripts_source_bucket}.")
    print (f"textract_source_bucket is {textract_source_bucket}.")
    print (f"query_staging_bucket is {query_staging_bucket}.")
    print (f"stock_data_source_bucket is {stock_data_source_bucket}.")
    print (f"multimodal_output_bucket is {multimodal_output_bucket}.")
else:
    print("Recheck our cloudformation stack name")

In [ ]:
kendra_client = boto3.client("kendra")

sync_response = kendra_client.start_data_source_sync_job(
    Id = kendra_data_source_id,
    IndexId = kendra_index_id
)

print("Wait for the data source to sync with the index.")

time.sleep(30)

while True:
    jobs = kendra_client.list_data_source_sync_jobs(
        Id = kendra_data_source_id,
        IndexId = kendra_index_id
    )

    status = jobs["History"][0]["Status"]
    print(" Syncing data source. Status: "+status)
    
    if status != "SYNCING":
        break
    time.sleep(30)


In [ ]:
from langchain.retrievers import AmazonKendraRetriever
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os
from langchain.chat_models import ChatAnthropic


def build_chain():
       
    retriever = AmazonKendraRetriever(index_id=kendra_index_id)

    prompt_template = """

      Human: This is a friendly conversation between a human and an AI. 
      The AI is talkative and provides specific details from its context but limits it to 240 tokens.
      If the AI does not know the answer to a question, it truthfully says it does not know.

      Net income can be net loss. If the value is in paranthesis, it means it is a loss and hence a negative value. For example, (1000) means -1000.
    
      
      Assistant: OK, got it, I'll be a talkative truthful AI assistant.

      Human: Here are a few documents: {context}
      Based on the above documents, provide a straightforward answer for {question}. 
      Answer "don't know" if not present in the document. 

      Assistant:"""
    
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    chain_type_kwargs = {"prompt": PROMPT}
 
    return RetrievalQA.from_chain_type(
        llm, 
        chain_type="stuff", 
        retriever=retriever, 
        chain_type_kwargs=chain_type_kwargs,
        return_source_documents=True
    )


def run_chain(prompt: str, history=[]):
    chain = build_chain()
    result = chain(prompt)
    return {
        "answer": result['result'],
        "source_documents": result['source_documents']
    }


In [ ]:
#Test the tool
result = run_chain("What's Amazon's total unearned revenue in 2022?")
result

# Define a toolkit

Now that we have defined all the individual tools, we will put together a toolkit for the agent.

In [ ]:
from langchain.agents.tools import Tool
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.tools.python.tool import PythonREPLTool

tools = [
    Tool(
        name="Stock Querying Tool",
        func=run_query,
        description="""
        Useful for when you need to answer questions about stocks. It only has information about stocks.
        """
    ),
    OptimizePortfolio(),
    Tool(
        name="Financial Information Lookup Tool",
        func=run_chain,
        description="""
        Useful for when you need to look up financial information using kendra. 
        """
    ),
    PythonREPLTool(),
    Tool(
        name="Sentiment Analysis Tool",
        func=SentimentAnalysis,
        description="""
        Useful for when you need to analyze the sentiment of a topic, such as "Return to Office".
        """
    ),
     Tool(
        name="Detect Phrases Tool",
        func=DetectKeyPhrases,
        description="""
        Useful for when you need to detect key phrases in recent quaterly reports.
        """
    ),
     Tool(
        name="Text Extraction Tool",
        func=IntiateTextExtractProcessing,
        description="""
        Useful for when you need to trigger conversion of  pdf version of quaterly reports to text files using amazon textextract
        """
    ),
     Tool(
        name="Transcribe Audio Tool",
        func=TranscribeAudio,
        description="""
        Useful for when you need to convert audio recordings of earnings calls from audio to text format using Amazon Transcribe
        """
    )
]

Modify the prompt template to provide the agent guidance on how to use the tools. The current template of prompt is based on Anthropic Claude2. If you choose to use a different foundation model, please try tweaking the template. It's just like you would convey the same idea in slightly different ways to different people. 

In [ ]:
combo_template = """
    
    Let's first understand the problem and devise a plan to solve the problem. 
    Please output the plan starting with the header 'Plan:' and then followed by a numbered list of steps. Do not use past conversation history when you are planning the steps.
    Please make the plan the minimum number of steps required to accurately complete the task. 
        
    
    These are guidance on when to use a tool to solve a task, follow them strictly:  
    
    - When you need to find stock information, use "Stock Querying Tool" tool, as it provides more accurate and relevant answers. Pay attention to the time period. DO NOT search for answers on the internet.
    
    - When you need to look up financial information, such as revenue and income, use the "Financial Information Lookup Tool". 
    
    - When you need to find the key phrases information from quaterly report then use Detect Phrases Tool to get the information about all key phrases and respond with key phrases relavent to the question.

    - When you need to provide an optimized stock portfolio based on stock names, use Portfolio Optimization Tool. The output is the percent of fund you should spend on each stock.
    
    - When you need to do maths calculations, use "PythonREPLTool()" which is based on the python programming language. Only provide the required numerical values to this tool and test, for e.g. "stock_prices": [25, 50, 75] only pass in [25, 50, 75] not the text "stock prices:"
    
    - When you need to analyze sentiment of a topic, use "Sentiment Analysis Tool".
    
    
    "Closing price" means the most recent stock price of the time period. 
    
    Alternatively, When you come across values required from income statement, look for values from Annual Income Statement stored in csv format. 
           
    Income can be a positive (profit) or negative value (loss). If the value is in parenthesis (), take it as negative value, which means it's a loss. E.g. for (1000), use -1000.
         
    When you have a question about calculating a ratio, figure out the formula for the calculation, and find the relevant financial information using the proper tool. Then use PythonREPLTool() tool for calculation.

    If you can't find the answer, say "I can't find the answer for this question."
    
    DO NOT CREATE STEPS THAT ARE NOT NEEDED TO SOLVE A TASK. 
    
    Once you have answers for the question, stop and provide the final answers. The final answers should be a combination of the answers to all the questions, not just the last one.
    
    Please make sure you have a plan to answer all the questions in the input, not just the last one. 
    Sometimes the two questions are related, sometimes they are not. So in the end, include the answers for both questions.
    Please use these to construct an answer to the question , as though you were answering the question directly. Ensure that your answer is accurate and doesn’t contain any information not directly supported by the summary and quotes.
    If there are no data or information in this document that seem relevant to this question, please just say "I can’t find any relevant quotes".
    """

Adding a conversation history element to the bot

In [ ]:
chat_history_table = 'SessionTable' # Name of dynamoDB Table for storing converstaions (prompts and answers)
  
chat_session_id = '0'
  
if chat_session_id == '0' :
    chat_session_id = str(uuid.uuid4())

print (chat_session_id)

chat_history_memory = DynamoDBChatMessageHistory(table_name=chat_history_table, session_id=chat_session_id)
memory = ConversationBufferMemory(memory_key="chat_history", chat_memory=chat_history_memory, return_messages=True)

The agent has two parts, a planner and an executor. The planner sets up the steps necessary to answer the questions, and the executor carries out the plans using the tools in the toolkit.

In [ ]:
planner = load_chat_planner(llm)

system_message_prompt = SystemMessagePromptTemplate.from_template(combo_template)
human_message_prompt = planner.llm_chain.prompt.messages[1]
planner.llm_chain.prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

executor = load_agent_executor(llm, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True, max_iterations=2)

# Ask the agent questions!

Please note that the results are non-deterministic becuase of the nature of Large Languaeg Models (LLM), so what you get each time can be different, and they can also be different from what are in the blog posts.

In [ ]:
output = agent("What are the closing prices of stocks AAAA, WWW, DDD in year 2018? Can you build an optimized portfolio using these three stocks? Please provide answers to both questions.")
output